## Task 1 

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [3]:
import nltk
import itertools
import time
 

In [4]:
lines = open("holbrook.txt").readlines()
data = []

#assert(data[2] == {
#    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
#    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
#    'indexes': [9]
#})

In [5]:
## function for taking one line at a time and return dictionary with original , corrected and indexes of the error

def sent_parser(line):
   
    corrected=[]
    original=[]
    c=0            ## counter for storing index values where pipe operator is present
    index=[]      ## list for storing the index value of the misspelled words
    for word in line.split():               ## for iterating through words in the line or sentence
        if '|' in word:              ## if '|' is present in the word then 
            index.append(c)            ## return index by the value of counter 'c' at which correction is present
            
                
            ind=word.index('|')        #storing the index value of '|' in 'ind' variable and indexing is done for the correct and incorrect word 
            corrected.append(word[ind+1:])   #and the value is stored in the corrected and incorrect list      
            original.append(word[:ind])

        else :
            
            #if no correction is required then add word directly to the the corrected and incorrect list without 
            #any processing 
            
            corrected.append(word)
            original.append(word)

        c+=1
    
    dict={'original':original,'corrected':corrected,'indexes':index}
    
    return dict

def doc_parser(filename):
    lines = open(filename).readlines()
    data = []
    
    for i in lines:
        data.append(sent_parser(i))
    return data


In [6]:
## Calling parser function and passing the 'holbrook.txt' file in the function

filename='holbrook.txt'

## parser function and saving the result in data variable

data=doc_parser(filename)


In [7]:
assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [8]:
test = data[:100]
train = data[100:]



In [9]:
### separating train correct data, test correct data and test original data in differenct lists

train_correct=[i['corrected'] for i in train]
test_correct=[i['corrected'] for i in test]
test_original=[i['original'] for i in test] 

## **Task 2** : 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [10]:
from collections import Counter

train_correct_words=list(itertools.chain(*train_correct))  ## list of all words in the train_correct 
x=' '.join(train_correct_words)     
lower=x.lower()                   
x_words=nltk.word_tokenize(lower)
freq=Counter(x_words)                 ##Freq has all the words with their frequency from train_correct
train_correct_uniq=list(freq.keys())  ##list of all correct words that are unique
total_correct=sum(freq.values()) ## total count of words in train_correct


In [11]:
def unigram(word):
    word=word.lower()
    dict_c=dict(freq)      ## dict of counter 'freq' created in previous cell which has the frequency of words 
                           ## occuring in train_correct each words occuring 
    return dict_c[word]
  
def prob(word):            ## function for calculating probability of the word
    
    probability=unigram(word)/total_correct
    return probability



In [12]:
# Test your code with the following
assert(unigram("me")==87)

## **Task 3** : 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [13]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [14]:
def get_candidates(token):
    distance={}    ## dictionary is initalized to store minimal edit distance as values and token as the key
       
     
    for i in train_correct_uniq:
        distance[i]=edit_distance(i,token)

    sort=sorted(distance.items(),key=lambda x:x[1])   ## the dict is sorted with the minimum distance in the first place
    min_dist=sort[0][1]      ## storing the mininum value of the distance from word to token
    sorted_dic=dict(sort)

    min_dist_words=[key for key,value in sorted_dic.items() if value==min_dist]  ## list of words with the minimum distance value 
    
    return min_dist_words

In [15]:
# Test your code as follows
assert(get_candidates("minde"))


## Task 4 :

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [17]:
def correct(sentence):
    correct_sent=[]
    probability={}
    for i in sentence:
        
        if i.lower() not in train_correct_uniq:
                
                return_word=get_candidates(i)           ## calling function get_candidates to return the words with minimal distance
                               
                for j in return_word:
                    probability[j]=prob(j)   ## prob function is called to return the prbability of the given word
                
                sorted_prob=sorted(probability.items(),key=lambda x:x[1],reverse=True) 
                predicted_word=sorted_prob[0][0]       ##the word with the maximum probbility is returned as predicted_word
                correct_sent.append(predicted_word)    

                              
        else:
            correct_sent.append(i)
            
    return correct_sent


In [18]:
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])

## **Task 5** : 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [19]:
def accuracy(test_data):
    c_correct=0
    ## list of sentences in the test-original
    test_original=[test_indi['original'] for test_indi in test_data]

    ## list of sentences in the test-correct
    test_correct=[test_indi['corrected'] for test_indi in test_data]
    total=len(list(itertools.chain(*test_correct)))
              
    for i in range(len(test_original)): 
        
        """predicted is the sentence predicted by the correct function """
        predict=correct(test_original[i])      
        for j in range(len(predict)):          ## each word in predict sentence is then compared with the test_correct word
                if predict[j]==test_correct[i][j]: 
                        c_correct +=1
        
        accuracy=(c_correct)/total
    
    return accuracy

In [20]:
%%time
acc=accuracy(test)
print('The accuracy for model is : ', round(acc,4))

The accuracy for model is :  0.8237
Wall time: 23 s


## **Task 6 :**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)

### Observations and modification to improve algorithm:

- **NNP tag recognition (Named entity)**: It is observed from the dataset that there are many proper noun present in the test data which are not present in train data. Therefore, the named entities are also getting converted to the nearest matched words from the unique words of the train_correct vocabulary. A list could be created that can capture the named entity in the sentence so that we can avoid processing it in the correct function and directly append it to the list. Ex. In the test data there are some words like 'Anglia' which is converting to 'English' and also from train_original "NIGEL THRUSH page 48" is converting "JILL THE page 48".<br>Following steps are taken to capture proper nouns:<br>
    1) the sentence is taken as input.<br>
    2) Pos_tag has been performed.<br>
    3) The keys with tags as 'NNP' are extracted from the sentence.<br>
<br>
- **Words.words()**: nltk corpus of words are used to check the whether the word is valid or not. If it is not valid then processing the word through correct function otherwise directly append it to the corrected word.
<br>

- **Not considering the numbers**: There are some numbers or digits which are converting to the most nearest number due to their non presence in the training set. I have tried not converting the strings that are number and pass them as it is in the corrected form.It is slighlty increasing the accuracy.
<br><br>
- **Bigram Probability**: Bigram probability (conditional probability) can be calculated for the words returned from get_candidates function and one with the highest bigram probability value. The candidate should be selected using the bigram language model. If the word is first word in the sentence then take the unigram probability otherwise go with calculating conditional bigram probability.
<br>

- **Good Turing Smoothing**: While calculating probability in case of bigram probability, if the word is not in the vocabulary, it can result in 0 probability. Therefore, smoothing techniques can be used to overcome this issue. Good turing can be helpful in place where the we have to calculate the probability of the unseen words.
<br>
- **Add one smoothing**: Add one smoothing was implemented for out of vocabulary words in the calculation of unigram probability. The result didn't help much though. the accuracy remained almost same.



In [21]:
"""Function created to capture the word that are proper nouns """

def named_en(sent):
    named_e=[]
    for i,j in nltk.pos_tag(sent):
        if j=='NNP':
            named_e.append(i)
    return named_e

## **Task 7 :**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

### Evaluation:

The accuracy has improved from 82.37 % to 89.99 % by making moditifications mentioned above in the previous algorithm. Implementing some changes are has increased the accuracy by 7.5 %.

In [22]:
from nltk.corpus import words

def correct_mod(sentence):
    correct_sent=[]
    probability={}
    named_ent=named_en(sentence)
    for i in sentence:
        
        """If the word is digit or is proper noun or present in train_correct vocabulary the it is not processed through correct 
        function and directly appended to the corrected sentence in correct_sent"""
        
        if i not in named_ent and not i.isdigit() and i not in words.words() and i.lower() not in train_correct_uniq:
            return_word=get_candidates(i)           
            for j in return_word:
                probability[j]=prob(j)   ## prob function is called to return the prbability of the given word

            sorted_prob=sorted(probability.items(),key=lambda x:x[1],reverse=True) 
            predicted_word=sorted_prob[0][0]       ##the word with the maximum probbility is returned as predicted_word
            correct_sent.append(predicted_word)    
        
        else:
            correct_sent.append(i)
            
    return correct_sent

In [23]:
def accuracy_mod(test_data):
    c_correct=0
    ## list of sentences in the test-original
    test_original=[test_indi['original'] for test_indi in test_data]

    ## list of sentences in the test-correct
    test_correct=[test_indi['corrected'] for test_indi in test_data]
    total=len(list(itertools.chain(*test_correct)))
              
    for i in range(len(test_original)):
        x=correct_mod(test_original[i])
        for j in range(len(x)):
                if x[j]==test_correct[i][j]:
                        c_correct +=1
        
        accuracy=(c_correct)/total
    
    return accuracy



In [ ]:
print('The modified accuracy is :',round(accuracy_mod(test),4))